## set enviromental variables

Make sure you set these two enviromental variables:

* `HOOD_PROJECT` should lead to the HOOD repository
* `HOOD_DATA` should lead to a data folder (see `README.md` for details)

In [ ]:
import os

HOOD_PROJECT = "/path/to/hood/repository"
HOOD_DATA = "/path/to/hood/data"

os.environ["HOOD_PROJECT"] = HOOD_PROJECT
os.environ["HOOD_DATA"] = HOOD_DATA


## Download Validation Sequences

First, you'll need to download the validation sequences along with some additional info.

Download [this archive](https://drive.google.com/file/d/1jFkDWPZW2HwYsYqcXAC3hX0NlumBnqT3/view?usp=drive_link) and extract it into `$HOOD_DATA/aux_data/validation_sequences`

## or create validation sequences yourself

You will still need to download and unpack data as described above

In [ ]:
from utils.make_comparison import create_sequences

model = 'cvpr' # 'postcvpr'|'cvpr'|'fine15'|'fine48'
comparisons = ['snug', 'ssch']
rollouts_dir = 'validation_sequences/my_hood_rollouts' # dir to save rollouts, relative to $HOOD_DATA/aux_data

for comparison in comparisons:
    create_sequences(model, comparison, rollouts_dir)

Alternatively, you can run 

```
python utils/make_comparison.py model=cvpr comparison=snug rollouts_dir=validation_sequences/my_hood_rollouts
```

# compute by-frame metrics

Once you have downloaded the validation sequences you can compute metrics for each frame in them. 

To do that, use `compute_and_store_metrics` function.

This function will build a `runner` object with a set of criterions according to the `configs/aux/metrics.yaml` configuraion file. Then it will construct a `Batch` object for each time step in the validation sequences and pass is through `runner.criterion_pass()` to compute metrics.

### Canonical geometries
Some of the criterions require a canonical geometry of the garment (e.g. stretching loss computes discrepancy betweet triangle areas between current and canonical geometry).

Since in our paper we compared our method with [SNUG](http://mslab.es/projects/SNUG/) and [SSCH](http://mslab.es/projects/SelfSupervisedGarmentCollisions/), we used the canonical geometries generated by these methods to compute metrics. These canonical geometries are stored in `$HOOD_DATA/aux_data/validation_sequences/rest_geometries`

### Fixed shape parameters
For evaluation we have sampled random shape parameters for SMPL from U(-2, 2) for each sequence and then kept them consistent across all compared methods. 

The randomly sampled beta parameters are stored in `$HOOD_DATA/aux_data/validation_sequences/betas.pkl`.

The `.csv` data splits used for validation sequences (`$HOOD_DATA/aux_data/validation_sequences/datasplits`) store an index of the shape parameters for each sequence that maps the sequences to the betas stored in `betas.pkl`


### Output files
Computed by-frame metrics are stored in `.pkl` files in `$HOOD_DATA/aux_data/validation_sequences/metrics/vs_X.pkl`, where `X` is either `snug` and `ssch` for metrics computed with canonical geometries from the corresponding method.

Further in this notebook we demonstrate how to display aggregated metrics used in Table 1 of the paper and Tables 1 and 2 of the supplementary material.


In [ ]:
from utils.arguments import load_params, create_modules
from utils.common import move2device, pickle_load
import torch, pickle
from collections import defaultdict
import numpy as np
from torch_geometric.data import Batch
import os
import pandas as pd
from tqdm import tqdm
from pathlib import Path

from utils.metrics import compute_and_store_metrics, print_metrics_vs_baselines, print_metrics_vs_sota

from utils.defaults import DEFAULTS

In [ ]:
seqs_root = Path(DEFAULTS.aux_data) / 'validation_sequences' / 'hood_rollouts' # in 'hood_rollouts' precomputed sequences are stored, to use sequences you generated, use directory you provided above
out_root = Path(DEFAULTS.aux_data) / 'validation_sequences' / 'metrics' 
canonicalpos_root = Path(DEFAULTS.aux_data) / 'validation_sequences' / 'rest_geometries' 
datasplit_root = Path(DEFAULTS.aux_data) / 'validation_sequences' / 'datasplits' 

compute_and_store_metrics(out_root, seqs_root, canonicalpos_root, datasplit_root)

# Print aggregeted metrics

## Table 1 of the main paper and Table 2 of the supplementary material

To produce the numbers that precisely match the paper, use parameter `match_paper=True` (it excludes one of the sequences from comparison, for which the baseline `Fine15` diverged).

To average the metrics across all sequences (e.g. if you want to compare your results to HOOD), use `match_paper=False`


In [ ]:
metrics_root = Path(DEFAULTS.aux_data) / 'validation_sequences' / 'metrics'
print_metrics_vs_baselines(metrics_root, match_paper=True)

## Table 1 of the supplementary material

In [ ]:
print_metrics_vs_sota(metrics_root)